# Part 08: Visualize seattle street groups by type
michael babb  
2024 11 24

In [ ]:
# standard
import os

In [ ]:
# external
import geopandas as gpd
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# custom
import run_constants as rc
from utils import *

# load the working seattle street network data

In [ ]:
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S03_SND_WORKING_IN_FILE_NAME)

In [ ]:
gdf = gpd.read_file(filename = fpn)

In [ ]:
gdf.shape

In [ ]:
gdf.columns

In [ ]:
gdf['ord_stname_concat'].unique().shape

In [ ]:
gdf['ord_stname_unique'].unique().shape

In [ ]:
gdf['city_sector'].unique()

## load the manually created street group data
These data were created in Step 6a, but we'll bring them into the Step 3 workflow.

In [ ]:
fpn = os.path.join(rc.INPUT_FILE_PATH, rc.S03_STREET_GROUP_IN_FILE_NAME)

In [ ]:
# ['sort_order', 'ord_street_name', 'ord_street_type', 'city_sector']
sg_df = pd.read_excel(io = fpn, index_col=[0, 1, 2, 3])
sg_df = sg_df.reset_index()

In [ ]:
# remove some columns
drop_cols = ['sort_order', 'progress', 'count']
sg_df = sg_df.drop(labels = drop_cols, axis = 1)


In [ ]:
test_join = pd.merge(left = gdf, right = sg_df, how = 'outer', indicator=True)
test_join['_merge'].value_counts()

In [ ]:
gdf = test_join.drop(labels = ['_merge'], axis = 1)

In [ ]:
gdf['group_id'] = gdf['group_id'].fillna(1).astype(int)
gdf.shape

# create an id across street groups

In [ ]:
gdf.head()

In [ ]:
gdf.columns

In [ ]:
col_names = ['ord_street_name', 'ord_street_type', 'ord_stname_concat',
             'ord_stname_unique', 'city_sector', 'group_id']
id_df = gdf[col_names].drop_duplicates()

In [ ]:
id_df['ord_stname_type'] = id_df['ord_street_name'] + ' ' + id_df['ord_street_type']

In [ ]:
id_df['sort_order'] = id_df['ord_street_name'].map(get_sort_order)

In [ ]:
id_df = id_df.sort_values(by = ['sort_order', 'ord_street_type', 'city_sector'])

In [ ]:
id_df.head()

In [ ]:
id_df['ord_stname_unique'].unique().shape

In [ ]:
# let's get the groups! can we aggregate this?

In [ ]:
col_names = ['sort_order', 'ord_street_name', 'ord_street_type', 'ord_stname_type', 'group_id', 'city_sector']
test_agg = id_df[col_names].groupby(col_names[:-1]).agg(city_sector_group = ('city_sector', lambda x: '_'.join(sorted(set(x)))),
                                                        n_groups = ('city_sector', lambda x: len(set(x)))).reset_index()

In [ ]:
test_agg.head()

In [ ]:
test_agg['city_sector_group'].unique().shape

In [ ]:
test_agg.head()

In [ ]:
# export to excel - this will be manually updated in conjunction with the plots
# generated below. This is necessary in order to better label street groups
# order the city sectors from top-to-bottom and left-to-right
city_sector_check = test_agg[['city_sector_group', 'ord_street_type', 'n_groups']].drop_duplicates().reset_index(drop = True)

In [ ]:
city_sector_check.head()

In [ ]:
city_sector_check.shape

In [ ]:
city_sector_check = city_sector_check.sort_values(by = ['city_sector_group', 'ord_street_type'])

In [ ]:
ofpn = os.path.join(rc.ANALYSIS_OUTPUT_FILE_PATH, rc.S08_CPG_OUTPUT_FILE_NAME)

In [ ]:
city_sector_check.to_excel(excel_writer=ofpn, index = False)

# join back to the working gdf

In [ ]:
gdf.columns

In [ ]:
test_join = pd.merge(left = gdf, right = test_agg, how = 'outer', indicator=True)

In [ ]:
test_join['_merge'].value_counts()

In [ ]:
test_join.shape

In [ ]:
test_join.columns

In [ ]:
gdf = test_join.drop(labels = '_merge', axis = 1)

# MAKE A PLOT OF THE DIFFERENT CITY GROUPS BY STREET TYPE

In [ ]:
# use the same colors for each city sector
cdm = {}
dir_list = ['CNTR', 'E', 'N', 'NE', 'NW', 'S', 'SW', 'W']
for idir, dir_value in enumerate(dir_list):
    cdm[dir_value] = mpl.colormaps["Dark2"].colors[idir]

my_cmap = mpl.colors.ListedColormap([cdm[c] for c in dir_list])

In [ ]:
wgdf = gdf.loc[gdf['n_groups'] > 1, :].copy()

In [ ]:
wgdf.shape

In [ ]:
col_names = ['city_sector_group', 'ord_street_type']
control_df = wgdf[col_names].drop_duplicates()

In [ ]:
control_df.shape

In [ ]:
control_df.head()

In [ ]:
# plot bounds to use across each plot
bounds = [-122.4197794277490061,47.4803548409661005, -122.2200188105690017,47.7341482423694004]

In [ ]:
# load the boundaries of the city sectors
ifpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S02_CITY_SECTORS_OUT_FILE_NAME)
cs_gdf = gpd.read_file(filename=ifpn)

In [ ]:
control_df.head()

In [ ]:
ost = 'BR'
pgdf = gdf.loc[(gdf['ord_street_type'] == ost), :]

In [ ]:
pgdf['city_sector_group'].unique()

In [ ]:
pgdf.head()

In [ ]:
make_plots = True
if make_plots:
    for cr, crow in control_df.iterrows():
        cpg = crow['city_sector_group']
        ost = crow['ord_street_type']

        output_file_name = f"{ost}_{cpg}.png"
        ofpn = os.path.join(rc.S08_PLOT_OUTPUT_FILE_PATH_CITY_SECTOR_GROUPS, output_file_name)
        print(ofpn)

        # now, let's make a map...
        pgdf = gdf.loc[(gdf['city_sector_group'] == cpg) &
                    (gdf['ord_street_type'] == ost), :]
        to_draw = pgdf[['city_sector', 'geometry']].dissolve(by = 'city_sector', as_index = False)
        #to_draw['coords'] = to_draw['geometry'].map(lambda x: x.centroid.coords[0])
        fig = plt.figure(layout = 'constrained', figsize = (5, 10))
        gs = GridSpec(1,1, figure = fig, height_ratios = [1])
        ax1 = fig.add_subplot(gs[0,0])
        ax1.set_xlim(bounds[0], bounds[2])
        ax1.set_ylim(bounds[1], bounds[3])
        cs_gdf.plot(ax = ax1, column = 'city_sector', alpha = .2)
        to_draw.plot(ax = ax1, column = 'city_sector', cmap = my_cmap,  linewidth = 5, legend = True)

        #for irrow, row in to_draw.iterrows():    
        #    ax1.annotate(text=row['city_sector'], xy=row['coords'], fontsize = 16 )

        ax1.set_axis_off()
        
        my_title = f"Street Type: {ost} | City Portion Group {cpg}"
        plt.title(label = my_title)

        #plt.show()

        fig.savefig(fname = ofpn)
        plt.close()